In [1]:
using Distributions
using Optim

In [2]:
function flowpayoffs(supportX,beta,delta)
    # Returns flowpayoffs from not serving (0) or serving (1)
    nSuppX = size(supportX,1);
    # u0 is (nSuppX,2), first column if previous A=0, second if previous A=1
    u0 = [zeros(nSuppX,1) -delta[1]*ones(nSuppX,1)];
    u1 = [ones(nSuppX,1) supportX]*beta*[1 1]-delta[2]*ones(nSuppX,1)*[1 0];
    return u0,u1;
end

flowpayoffs (generic function with 1 method)

In [3]:
function bellman(capU0,capU1,u0,u1,capPi,rho)
    r0 = log(exp(capU0[:,1])+exp(capU1[:,1]));
    r1 = log(exp(capU0[:,2])+exp(capU1[:,2]));
    # Then, it applies (\ref{eq:bellman}) to compute new values of |capU0| and |capU1|
    capU0 = u0+rho*capPi*r0*[1 1];
    capU1 = u1+rho*capPi*r1*[1 1];
    # Here, the conditional expectation over $X_{t+1}$ in (\ref{eq:bellman}) is taken by premultiplying the vectors
    # |r0| and |r1| by the Markov transition matrix |capPi|. The vectors |r0| and |r1| are postmultiplied by |[1 1]|
    # because the surpluses, and therefore the continuation payoffs, are independent of the past choice that indexes
    # the columns of |capU0| and |capU1|.
    # The logit assumption only affects the operator $\Psi$, and therefore the function |bellman|, through the
    # specification of the surpluses $R_0$ and $R_1$ in (\ref{eq:surplus}). If you want to change the logit assumption,
    # you should change the computation of |r0| and |r1| (and make sure to adapt the computation of choice probabilities
    # and inverse choice probabilities elsewhere as well).
    return capU0,capU1;
end

bellman (generic function with 1 method)

In [4]:
function fixedPoint(u0,u1,capPi,rho,tolFixedPoint)
    nSuppX = size(capPi,1);
    capU0 = zeros(nSuppX,2);
    capU1 = zeros(nSuppX,2);
    
    inU0 = capU0+2*tolFixedPoint;
    inU1 = capU1+2*tolFixedPoint;
    
    while (maximum(maximum(abs(inU0-capU0)))>tolFixedPoint) || (maximum(maximum(abs(inU1-capU1)))>tolFixedPoint)
        inU0 = capU0;
        inU1 = capU1;
        capU0,capU1 = bellman(inU0,inU1,u0,u1,capPi,rho);
    end
    return capU0,capU1;
end

fixedPoint (generic function with 1 method)

In [5]:
function randomDiscrete(p)
    nSupp = size(p,1);
    nVar = size(p,2);
    uniformDraws = ones(nSupp-1,1)*rand(1,nVar);
    cumulativeP = cumsum(p);
    return y = sum([ones(1,nVar);cumulativeP[1:nSupp-1,:].<uniformDraws],1);
end

randomDiscrete (generic function with 1 method)

In [6]:
function simulateData(deltaU,capPi,nPeriods,nFirms)
    nSuppX = size(capPi,1);
    oneMinPi = eye(nSuppX)-capPi';
    pInf = [oneMinPi[1:nSuppX-1,:];ones(1,nSuppX)]\[zeros(nSuppX-1,1);1];
    # Then, it uses the auxiliary function |randomDiscrete| (see Appendix \ref{misc}) and the values stored in |pInf| to
    # simulate a $1\times N$ vector of values of $X_1$ from the stationary distribution $P^\infty$ and stores their
    # indices in |iX|.
    iX = randomDiscrete(pInf*ones(1,nFirms));
    # Using these $N$ simulated values of $X_1$, and $N$ simulated values of $-\Delta\varepsilon_1\equiv\varepsilon_1(0)-\varepsilon_1(1)$
    # that are stored in |deltaEpsilon|, it simulates $N$ values of the first choice by using that
    # $A_1=1$ if $\Delta U(X_1,0)>-\Delta\varepsilon_1$ and $A_1=0$ otherwise. These are stored in the $1\times N$ vector |choices|.
    deltaEpsilon = rand(Gumbel(),1,nFirms)-rand(Gumbel(),1,nFirms);
    choices = deltaU[iX,1]' .> deltaEpsilon;
    # Finally, $N$ values of $X_t$ are simulated, using the transition matrix $\Pi$ and |randomDiscrete|, and their
    # indices added as a row to the bottom of the $(t-1)\times N$ matrix |iX|; and $N$ values of $A_t$ are simulated,
    # using that  $A_t=1$ if $\Delta U(X_t,A_{t-1})>-\Delta\varepsilon_t$ and $A_t=0$ otherwise, and stored as a row at
    # the bottom of the $(t-1)\times N$ matrix |choices|; recursively for $t=2,\ldots,T$.
    for t = 2:nPeriods
        iX = [iX;randomDiscrete(capPi[iX[end,:],:]')];
        deltaEpsilon = rand(Gumbel(),1,nFirms)-rand(Gumbel(),1,nFirms);
        choices = [choices;(deltaU[iX[end,:]+nSuppX*choices[end,:]]' .> deltaEpsilon)];
    end
    return choices,iX;
end

simulateData (generic function with 1 method)

In [7]:
function estimatePi(iX,nSuppX)
    nPeriods = size(iX,1);
    piHat = zeros(nSuppX,nSuppX);
    # Then, for each pair $(i,j)\in\{1,\ldots,K\}\times\{1,\ldots,K\}$, it estimates the probability $\Pi_{ij}=\Pr(X_{t+1}=x^j|X_t=x^i)$
    # by the appropriate sample frequency, the number of transitions from $i$ to $j$ divided by the total number of
    # transitions from $i$ in the data |iX|.
    for i=1:nSuppX
        for j=1:nSuppX
            piHat[i,j] = sum(sum((iX[2:nPeriods,:].==j)&(iX[1:nPeriods-1,:].==i)))/sum(sum((iX[1:nPeriods-1,:].==i)));
        end
    end
    # Note that |estimatePi| requires a positive number of transition observations from each state. More generally, the
    # frequency estimator that it implements only performs well with samples that are large relative to the state space.
    # With relatively small samples, the frequency estimator should be replaced by one that smoothes across support points.
    return piHat;
end

estimatePi (generic function with 1 method)

In [8]:
function negLogLik(pars::Vector)
    beta = pars[1:2];        # beta0hat seems always to be biased towards -1
    delta = [0;pars[3]];     # delta0 assumed known, the two deltahats are biased otherwise
    rho = 0.95;              # Assumed known, optimization unstable otherwise
    capPi = piHat;
    nSuppX = size(supportX,1);
    # Next, it computes the flow payoffs $u_0$ (|u0|) and $u_1$ (|u1|), the choice-specific net expected discounted values
    # $U_0$ (|capU0|) and $U_1$ (|capU1|), their contrast $\Delta U$ (|deltaU|), and the implied probabilities $1/\left[1+\exp(\Delta U)\right]$
    # of not serving the market (|pExit|) for the inputted parameter values. Note that this implements the NFXP procedure's
    # inner loop.
    u0,u1 = flowpayoffs(supportX,beta,delta);
    
    capU0,capU1 = fixedPoint(u0,u1,capPi,rho,tolFixedPoint);
    deltaU = capU1-capU0;
    pExit = 1./(1+exp(deltaU));
    
    laggedChoices = [zeros(1,size(choices,2));choices[1:end-1,:]];
    p = choices + (1-2*choices).*reshape(pExit[iX+nSuppX*laggedChoices],nPeriods,nFirms);
    nll = -sum(log(p));
    
    println("Negative log likelihood = $nll, pars= $pars");
    return nll;
end

negLogLik (generic function with 1 method)

In [9]:
function negScore!(pars::Vector)
    beta = pars[1:2];
    delta = [0.0;pars[3]];
    rho = 0.95;     # Assumed known
    capPi = piHat;
    nSuppX = size(supportX,1);
    # Next, it computes the flow payoffs $u_0$ (|u0|) and $u_1$ (|u1|), the choice-specific net expected discounted values
    # $U_0$ (|capU0|) and $U_1$ (|capU1|), their contrast $\Delta U$ (|deltaU|), and the implied probabilities $1/\left[1+\exp(\Delta U)\right]$
    # of not serving the market (|pExit|) for the inputted parameter values. Note that this implements the NFXP procedure's
    # inner loop.
    u0,u1 = flowpayoffs(supportX,beta,delta);
    capU0,capU1 = fixedPoint(u0,u1,capPi,rho,tolFixedPoint);
    deltaU = capU1-capU0;
    pExit = 1./(1+exp(deltaU));
    
    laggedChoices = [zeros(1,size(choices,2));choices[1:end-1,:]];
    p = choices + (1-2*choices).*reshape(pExit[iX+nSuppX*laggedChoices],nPeriods,nFirms);
    nll = -sum(log(p));
    
    # This calculates the Score
    d00 = rho*capPi*diagm(pExit[:,1]);
    d01 = rho*capPi*diagm(pExit[:,2]);
    d10 = rho*capPi-d00;
    d11 = rho*capPi-d01;
    dPsi_dUbar = [[d00;d00;zeros(2*nSuppX,nSuppX)] [zeros(2*nSuppX,nSuppX);d01;d01] [d10;d10;zeros(2*nSuppX,nSuppX)] [zeros(2*nSuppX,nSuppX);d11;d11]];
    
    dPsi_dTheta = [[zeros(2*nSuppX,1);ones(2*nSuppX,1)] [zeros(2*nSuppX,1);supportX;supportX] [zeros(2*nSuppX,1);-ones(nSuppX,1);zeros(nSuppX,1)]];
    
    # Next, it computes $\partial\bar U/\partial\theta'$ (|dUbar_dTheta|) and $\partial\Delta U/\partial\theta'$ (|dDeltaU_dTheta|).
    dUbar_dTheta   = (eye(4*nSuppX)-dPsi_dUbar)\dPsi_dTheta;
    dDeltaU_dTheta    = dUbar_dTheta[2*nSuppX+1:4*nSuppX,:]-dUbar_dTheta[1:2*nSuppX,:];
    
    # Finally, it computes the $1\times 3$ vector $-\partial\log\left[\prod_{t=1}^T p(a_{tn}|x_{tn},a_{(t-1)n})\right]/\partial \theta'$
    # for each $n$, stacks these individual (minus) score contributions in the $N\times 3$ matrix |negFirmScores|, and
    # sums them to compute minus the score vector, |negScore|.
    nTheta = size(dUbar_dTheta,2);
    tempScores = repmat((1-2*choices).*(1-p),1,nTheta);
    negFirmScores = reshape(tempScores,nPeriods,nFirms,nTheta);
    for i=1:nTheta
        negFirmScores[:,:,i] = negFirmScores[:,:,i].*reshape(dDeltaU_dTheta[iX+nSuppX*laggedChoices+2*(i-1)*nSuppX],nPeriods,nFirms);
    end
    negFirmScores = squeeze(sum(negFirmScores,1),1);
    negScore = sum(negFirmScores,1)';
    grad = squeeze(negScore,2);
    return grad;
end

negScore! (generic function with 1 method)

In [10]:
function info!(pars::Vector)
    beta = pars[1:2];
    delta = [0.0;pars[3]];
    rho = 0.95;     # Asummed known
    capPi = piHat;
    nSuppX = size(supportX,1);
    # Next, it computes the flow payoffs $u_0$ (|u0|) and $u_1$ (|u1|), the choice-specific net expected discounted values
    # $U_0$ (|capU0|) and $U_1$ (|capU1|), their contrast $\Delta U$ (|deltaU|), and the implied probabilities $1/\left[1+\exp(\Delta U)\right]$
    # of not serving the market (|pExit|) for the inputted parameter values. Note that this implements the NFXP procedure's
    # inner loop.
    u0,u1 = flowpayoffs(supportX,beta,delta);
    capU0,capU1 = fixedPoint(u0,u1,capPi,rho,tolFixedPoint);
    deltaU = capU1-capU0;
    pExit = 1./(1+exp(deltaU));
    
    laggedChoices = [zeros(1,size(choices,2));choices[1:end-1,:]];
    p = choices + (1-2*choices).*reshape(pExit[iX+nSuppX*laggedChoices],nPeriods,nFirms);
    nll = -sum(log(p));
    
    # This calculates the Score
    d00 = rho*capPi*diagm(pExit[:,1]);
    d01 = rho*capPi*diagm(pExit[:,2]);
    d10 = rho*capPi-d00;
    d11 = rho*capPi-d01;
    dPsi_dUbar = [[d00;d00;zeros(2*nSuppX,nSuppX)] [zeros(2*nSuppX,nSuppX);d01;d01] [d10;d10;zeros(2*nSuppX,nSuppX)] [zeros(2*nSuppX,nSuppX);d11;d11]];
    
    dPsi_dTheta = [[zeros(2*nSuppX,1);ones(2*nSuppX,1)] [zeros(2*nSuppX,1);supportX;supportX] [zeros(2*nSuppX,1);-ones(nSuppX,1);zeros(nSuppX,1)]];
    
    # Next, it computes $\partial\bar U/\partial\theta'$ (|dUbar_dTheta|) and $\partial\Delta U/\partial\theta'$ (|dDeltaU_dTheta|).
    dUbar_dTheta = (eye(4*nSuppX)-dPsi_dUbar)\dPsi_dTheta;
    dDeltaU_dTheta = dUbar_dTheta[2*nSuppX+1:4*nSuppX,:]-dUbar_dTheta[1:2*nSuppX,:];
    
    # Finally, it computes the $1\times 3$ vector $-\partial\log\left[\prod_{t=1}^T p(a_{tn}|x_{tn},a_{(t-1)n})\right]/\partial \theta'$
    # for each $n$, stacks these individual (minus) score contributions in the $N\times 3$ matrix |negFirmScores|, and
    # sums them to compute minus the score vector, |negScore|.
    nTheta = size(dUbar_dTheta,2);
    tempScores = repmat((1-2*choices).*(1-p),1,nTheta);
    negFirmScores = reshape(tempScores,nPeriods,nFirms,nTheta);
    for i=1:nTheta
        negFirmScores[:,:,i] = negFirmScores[:,:,i].*reshape(dDeltaU_dTheta[iX+nSuppX*laggedChoices+2*(i-1)*nSuppX],nPeriods,nFirms);
    end
    negFirmScores = squeeze(sum(negFirmScores,1),1);
    informationMatrix = zeros(nTheta,nTheta);
    for n=1:size(negFirmScores,1)
        informationMatrix = informationMatrix + negFirmScores[n,:]'*negFirmScores[n,:];
    end
    return hess=informationMatrix;
end

info! (generic function with 1 method)

In [11]:
nPeriods = 100;
nFirms = 1000;

In [12]:
tolFixedPoint = 1e-10;

In [13]:
nSuppX = 5;
supportX = (1:nSuppX);
capPi = 1./(1+abs(ones(nSuppX,1)*collect((1:nSuppX))'-collect(1:nSuppX)*ones(1,nSuppX)));
capPi = capPi./(sum(capPi,1)'*ones(1,nSuppX));
beta = [-0.1*nSuppX;0.2];
delta = [0;1];
rho = 0.95;

In [14]:
u0,u1 = flowpayoffs(supportX,beta,delta);
capU0,capU1 = fixedPoint(u0,u1,capPi,rho,tolFixedPoint);
deltaU = capU1-capU0;

In [15]:
choices,iX = simulateData(deltaU,capPi,nPeriods,nFirms);

In [16]:
piHat = estimatePi(iX,nSuppX);

output = optimize(negLogLik,[-1.0;-0.1;0.5],method=:nelder_mead,ftol=1e-6,xtol=1e-10);

startvalues = output.initial_x;
maxLikEstimates = output.minimum;

hessian = info!(maxLikEstimates);
standardErrors = sqrt(diag(inv(hessian)));

Negative log likelihood = 70696.9498578224, pars= [0.0,-0.1,0.5]
Negative log likelihood = 99385.97604589684, pars= [-1.0,0.9,0.5]
Negative log likelihood = 105723.87852760764, pars= [-1.0,-0.1,1.5]
Negative log likelihood = 95884.54810607055, pars= [-1.0,-0.1,0.5]
Negative log likelihood = 86655.30057261164, pars= [-0.33333333333333326,0.5666666666666668,-0.5]
Negative log likelihood = 123354.69282851089, pars= [0.11111111111111116,-0.6555555555555554,-0.16666666666666669]
Negative log likelihood = 74084.7009949316, pars= [-0.7222222222222222,0.5111111111111112,0.3333333333333333]
Negative log likelihood = 122096.4642837309, pars= [0.2962962962962964,0.751851851851852,-0.2777777777777778]
Negative log likelihood = 69339.79383769154, pars= [-0.6759259259259259,0.112962962962963,0.3055555555555555]
Negative log likelihood = 102434.71310313372, pars= [-0.5987654320987655,-0.21728395061728395,1.259259259259259]
Negative log likelihood = 72645.73772848, pars= [-0.39969135802469136,0.370679

In [17]:
println("=========================");
println(output);
println("=========================");
println("Negative Score");
println(negScore!(maxLikEstimates));
println("=========================");
println("Information Matrix");
println(hessian);
println("=========================");
println("Summary of Results");
println("--------------------------------------------");
println(" True  Start     estim              ste.");
println([[beta;delta[2]] startvalues maxLikEstimates standardErrors]);


Results of Optimization Algorithm
 * Algorithm: Nelder-Mead
 * Starting Point: [-1.0,-0.1,0.5]
 * Minimum: [-0.5309650252535212,0.20847841692536484, ...]
 * Value of Function at Minimum: 64964.664053
 * Iterations: 69
 * Convergence: true
   * |x - x'| < NaN: false
   * |f(x) - f(x')| / |f(x)| < 1.0e-06: true
   * |g(x)| < NaN: false
   * Exceeded Maximum Number of Iterations: false
 * Objective Function Calls: 132
 * Gradient Call: 0
Negative Score
[0.022281605960635886,-0.07438424302769953,0.06648732921969236]
Information Matrix
[37624.45286662701 113159.37900456687 -93.7184090337311
 113159.37900456687 388973.80168778484 359.03631646905944
 -93.7184090337311 359.03631646905944 5415.13469771249]
Summary of Results
--------------------------------------------
 True  Start     estim              ste.
[-0.5 -1.0 -0.5309650252535212 0.014590955227388914
 0.2 -0.1 0.20847841692536484 0.004537979963937957
 1.0 0.5 0.9905238465255334 0.013600149409701686]
